In [ ]:
import requests
import os
from pprint import pprint

token = os.environ.get("OIDC_TOKEN")

if not token:
    raise Exception("Token environment variable 'OIDC_TOKEN' not set, obtain the value for it by launching a service (Jupyterlab or VSCode) at https://lab.dapla.ssb.no and running command 'echo $OIDC_TOKEN'. Then run 'export OIDC_TOKEN=<value obtained from Dapla Lab>' on this machine.")

# Only available from Statistics Norway's networks, requires authorization
INTERNAL_SERVER = "https://metadata.intern.test.ssb.no"

auth = {"Authorization": f"Bearer {token}"}

response = requests.get(f"{INTERNAL_SERVER}/variable-definitions", headers=auth)
response.raise_for_status()

pprint(response.content)